<a href="https://colab.research.google.com/github/JimKing100/nfl-test/blob/master/predictions/Prediction_Offense_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Installs
!pip install pmdarima

In [0]:
# Imports
import numpy as np
import pandas as pd
from statsmodels.tsa.arima_model import ARIMA
import pmdarima as pm
from sklearn import preprocessing

In [0]:
# Import data
original_df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/nfl-test/master/data-actuals/actuals_offense.csv')
kickers_df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/nfl-test/master/data-revised/rookies_non_kicker.csv')
offense_df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/nfl-test/master/data-revised/rookies_non_offense.csv')
player_df = pd.concat([kickers_df, offense_df], ignore_index=True)

In [4]:
# The dataframe of actual offensive points for each game from 2000-2019
original_df.head()

,player,first,last,name,position1,2000-game1,2000-game2,2000-game3,2000-game4,2000-game5,2000-game6,2000-game7,2000-game8,2000-game9,2000-game10,2000-game11,2000-game12,2000-game13,2000-game14,2000-game15,2000-game16,2001-game1,2001-game2,2001-game3,2001-game4,2001-game5,2001-game6,2001-game7,2001-game8,2001-game9,2001-game10,2001-game11,2001-game12,2001-game13,2001-game14,2001-game15,2001-game16,2002-game1,2002-game2,2002-game3,...,2017-game9,2017-game10,2017-game11,2017-game12,2017-game13,2017-game14,2017-game15,2017-game16,2018-game1,2018-game2,2018-game3,2018-game4,2018-game5,2018-game6,2018-game7,2018-game8,2018-game9,2018-game10,2018-game11,2018-game12,2018-game13,2018-game14,2018-game15,2018-game16,2019-game1,2019-game2,2019-game3,2019-game4,2019-game5,2019-game6,2019-game7,2019-game8,2019-game9,2019-game10,2019-game11,2019-game12,2019-game13,2019-game14,2019-game15,2019-game16
0,AD-0100,Andy,Dalton,Andy Dalton,QB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,15.6,18.56,19.16,19.26,7.64,0.52,11.42,21.68,16.52,26.60,14.08,24.78,12.92,17.16,7.92,20.20,6.92,19.34,8.10,0.00,0.00,0.00,0.00,0.00,21.72,19.64,16.06,3.64,19.98,13.60,18.34,17.16,0.00,0.00,0.00,13.32,9.58,2.14,33.84,16.60
1,AM-1150,A.J.,McCarron,A.J. McCarron,QB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.00,0.00,0.00,1.88,0.76,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.32,0.00,0.00,-0.20,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,17.90
2,AR-1300,Aaron,Rodgers,Aaron Rodgers,QB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.00,0.00,0.00,0.00,21.90,0.00,0.00,25.94,16.04,20.90,15.02,28.68,28.40,16.94,19.16,18.06,22.58,11.92,15.32,20.24,13.46,43.88,1.04,12.92,14.36,14.30,25.48,9.42,18.32,44.76,28.10,12.94,10.02,9.46,28.12,11.40,14.42,9.34,19.02
3,AT-0160,Alex,Tanney,Alex Tanney,QB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.04,0.00,0.00
4,BA-0375,Brandon,Allen,Brandon Allen,QB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,18.02,13.40,2.08,0.00,0.00,0.00,0.00,0.00


In [5]:
# The dataframe of all 2019 offensive players (kickers and offense)
player_df.head(50)

,player,first,last,name,position1,start
0,AV-0400,Adam,Vinatieri,Adam Vinatieri,K,1996
1,MB-4600,Matt,Bryant,Matt Bryant,K,2002
2,RG-1500,Robbie,Gould,Robbie Gould,K,2005
3,MN-0800,Mike,Nugent,Mike Nugent,K,2005
4,SG-0800,Stephen,Gostkowski,Stephen Gostkowski,K,2006
5,MP-2100,Matt,Prater,Matt Prater,K,2006
6,MC-3000,Mason,Crosby,Mason Crosby,K,2007
7,NF-0300,Nick,Folk,Nick Folk,K,2007
8,SH-0400,Stephen,Hauschka,Stephen Hauschka,K,2008
9,RS-3400,Ryan,Succop,Ryan Succop,K,2009


In [0]:
# Add a row to the final_df dataframe
# Each row represents the predicted points for each team
def add_row(df, p, f, l, n, pos, pred, act):

    df = df.append({'player': p,
                    'first': f,
                    'last': l,
                    'name': n,
                    'position1': pos,
                    'week1-prediction': pred,
                    'week1-actual': act
                    }, ignore_index=True)
    
    return df

In [16]:
# The main code for iterating through the player(offense and kicker) list, calculating the points and adding the rows
# to the final_df dataframe.
column_names = ['player',
                'first',
                'last',
                'name',
                'position1', 
                'week1-prediction',
                'week1-actual'
                 ]

player_list = offense_df['player'].tolist()

final_df = pd.DataFrame(columns = column_names)

for player in player_list:

    first = player_df['first'].loc[(player_df['player']==player)].iloc[0]
    last = player_df['last'].loc[(player_df['player']==player)].iloc[0]
    name = player_df['name'].loc[(player_df['player']==player)].iloc[0]
    position1 = player_df['position1'].loc[(player_df['player']==player)].iloc[0]
    start_year = player_df['start'].loc[(player_df['player']==player)].iloc[0]
    row = original_df.index[(original_df['player']==player)][0]

    if start_year < 2000:
        start_year = 2000
    col = ((start_year - 2000) * 16) + 5
    train_data = original_df.iloc[row, col:309]
    actuals = original_df.iloc[row, 309:325]
    act_points = actuals.sum()

    print(player)

    if (player != 'GG-0310') & (player != 'KA-0737') & (player != 'JM-6775') & \
       (player != 'AL-0387') & (player != 'JW-5475'):
        # ARIMA model
        model = pm.auto_arima(train_data, start_p=1, start_q=1,
                              test='adf',       # use adftest to find optimal 'd'
                              max_p=3, max_q=3, # maximum p and q
                              m=1,              # frequency of series
                              d=None,           # let model determine 'd'
                              seasonal=False,   # No Seasonality
                              start_P=0, 
                              D=0, 
                              trace=False,
                              error_action='ignore',  
                              suppress_warnings=True, 
                              stepwise=True)
    
        # Forecast
        n_periods = 16
        fc = model.predict(n_periods=n_periods, return_conf_int=False)
        index_of_fc = np.arange(len(train_data), len(train_data)+n_periods)
        fc_series = pd.Series(fc, index=index_of_fc)
        pred_points = fc_series.sum()
    else:
        pred_points = 0

    final_df = add_row(final_df, player, first, last, name, position1, pred_points, act_points)

TB-2300
DB-3800
JM-2900
BR-1100
EM-0200
MS-0200
PR-0300
AR-1300
RF-0500
MM-4700
JF-1900
MR-2500
BH-1900
CD-0300
MS-4100
CM-1500
AD-0100
CN-0500
TT-0500
AT-0160
CK-0250
KC-2350
NF-0250
RG-1850
RT-0150
RW-3850
MB-0450
MG-0850
RG-1885
AM-1150


/usr/local/lib/python3.6/dist-packages/pmdarima/arima/auto.py:328: UserWarning: Input time-series is completely constant; returning a (0, 0, 0) ARMA.
  warnings.warn('Input time-series is completely constant; '


BB-2425
DC-0725
GG-0310
JG-0650
TB-2650
BH-2300
JW-5650
MM-0912
SM-0425
TS-1450
BA-0375
CW-1425


/usr/local/lib/python3.6/dist-packages/pmdarima/arima/auto.py:328: UserWarning: Input time-series is completely constant; returning a (0, 0, 0) ARMA.
  warnings.warn('Input time-series is completely constant; '


DP-2037
JB-5725
JD-2850
JG-1850
DW-1725
MT-1800
NM-0687
PM-0025
TH-1625
BM-0650
JA-1087
JR-3250
KA-0737
LF-0025
LJ-0150


...

JW-1050
JW-5475
KC-2375
KH-2025
MA-1287
MG-0175
MT-2350
MV-0012
RF-1075
RG-0012
RG-0025


/usr/local/lib/python3.6/dist-packages/pmdarima/arima/auto.py:328: UserWarning: Input time-series is completely constant; returning a (0, 0, 0) ARMA.
  warnings.warn('Input time-series is completely constant; '


RJ-0450
SM-2825
TQ-0100


/usr/local/lib/python3.6/dist-packages/pmdarima/arima/auto.py:328: UserWarning: Input time-series is completely constant; returning a (0, 0, 0) ARMA.
  warnings.warn('Input time-series is completely constant; '


TS-1425
TS-2425
VS-0175


In [0]:
# The final_df dataframe
final_df['week1-pred-rank'] = final_df['week1-prediction'].rank(ascending=False)
final_df['week1-act-rank'] = final_df['week1-actual'].rank(ascending=False)
final_df['week1-diff'] = final_df['week1-prediction'] - final_df['week1-actual']
final_df['week1-prediction'] = final_df['week1-prediction'].astype(int)
final_df['week1-diff'] = final_df['week1-diff'].astype(int)

In [18]:
# Calculate the metrics
final_df['pred-percent'] = final_df['week1-diff']/final_df['week1-prediction']
pred_median_error = final_df['pred-percent'].median()

print('Median Error - %.4f%%' % (pred_median_error * 100))

Median Error - 28.5714%


In [19]:
final_df.head(50)

,player,first,last,name,position1,week1-prediction,week1-actual,week1-pred-rank,week1-act-rank,week1-diff,pred-percent
0,TB-2300,Tom,Brady,Tom Brady,QB,261,265.68,33.0,14.0,-3,-0.011494
1,DB-3800,Drew,Brees,Drew Brees,QB,258,227.76,36.0,25.0,30,0.116279
2,JM-2900,Josh,McCown,Josh McCown,QB,58,0.76,179.0,440.0,57,0.982759
3,BR-1100,Ben,Roethlisberger,Ben Roethlisberger,QB,237,12.74,45.0,313.0,224,0.945148
4,EM-0200,Eli,Manning,Eli Manning,QB,226,56.38,46.0,175.0,169,0.747788
5,MS-0200,Matt,Schaub,Matt Schaub,QB,0,32.90,419.0,232.0,-32,-inf
6,PR-0300,Philip,Rivers,Philip Rivers,QB,275,240.50,28.0,22.0,35,0.127273
7,AR-1300,Aaron,Rodgers,Aaron Rodgers,QB,295,282.38,25.0,10.0,13,0.044068
8,RF-0500,Ryan,Fitzpatrick,Ryan Fitzpatrick,QB,120,242.46,108.0,20.0,-122,-1.016667
9,MM-4700,Matt,Moore,Matt Moore,QB,27,45.26,247.0,202.0,-17,-0.629630


In [0]:
# Save the results to .csv file
final_df.to_csv('/content/week1-pred-offense-norookies.csv', index=False)